# AWS Postgres RDS Debug Notebook

This notebook allows to run queries manually against the AWS postgres DB.

In [41]:
import sys
sys.path.insert(0, '/Users/ferdi/Documents/agent-copilot/src')

import psycopg2
import psycopg2.extras
from urllib.parse import urlparse
import pandas as pd
from datetime import datetime
import os

# Import the database module
from aco.server.database_manager import DB
DB.switch_mode("remote")

2025-11-19 21:15:04,869 - ACO - INFO - Switched to remote PostgreSQL database
2025-11-19 21:15:04,870 - ACO - DEBUG - Cleared SQLite connection cache
2025-11-19 21:15:04,871 - ACO - DEBUG - Closed PostgreSQL connection pool
2025-11-19 21:15:04,871 - ACO - DEBUG - Cleared PostgreSQL connection pool


## List Table Entries

### experiments table

In [39]:
# Get all experiments
experiments = DB.query_all(
    "SELECT session_id, parent_session_id, name, timestamp, success, notes FROM experiments ORDER BY timestamp DESC LIMIT 20"
)

if experiments:
    df_experiments = pd.DataFrame(experiments)
    print(f"Found {len(experiments)} experiments:")
    display(df_experiments)
else:
    print("No experiments found in database")

2025-11-19 15:38:18,422 - ACO - INFO - [QUERY_ALL] START thread=140704361521024 sql=SELECT session_id, parent_session_id, name, timestamp, success, notes FROM experiments ORDER BY time...
2025-11-19 15:38:18,722 - ACO - DEBUG - Database schema initialized
2025-11-19 15:38:18,724 - ACO - INFO - Initialized PostgreSQL connection pool to workflow-postgres.cm14iy6021bi.us-east-1.rds.amazonaws.com
2025-11-19 15:38:18,725 - ACO - INFO - [CONN] GET conn=5044824304 thread=140704361521024 caller=postgres.py:query_all:226
2025-11-19 15:38:18,757 - ACO - INFO - [QUERY_ALL] SUCCESS thread=140704361521024 conn=5044824304 rows=5
2025-11-19 15:38:18,758 - ACO - INFO - [CONN] RETURN conn=5044824304 thread=140704361521024 caller=postgres.py:query_all:238


Found 5 experiments:


,0,1,2,3,4,5
0,a120123d-1583-4ae9-aa7b-2d141f2179ec,a120123d-1583-4ae9-aa7b-2d141f2179ec,Workflow run,2025-11-19 15:37:41.065741,,Take notes.
1,abc4734a-1caf-4a77-813e-7551e013dfb5,abc4734a-1caf-4a77-813e-7551e013dfb5,Workflow run,2025-11-19 15:35:07.605128,,Take notes.
2,26f7166b-fc4f-42a0-b565-262a6c762908,26f7166b-fc4f-42a0-b565-262a6c762908,Workflow run,2025-11-19 15:31:54.964566,,Take notes.
3,125f3b27-64b8-44b8-82f6-d5bb946fe40b,125f3b27-64b8-44b8-82f6-d5bb946fe40b,Workflow run,2025-11-19 15:21:23.315116,,Take notes.
4,692b7175-b378-4c58-9aaa-c16c38decfb6,692b7175-b378-4c58-9aaa-c16c38decfb6,Workflow run,2025-11-19 14:56:09.598512,,Take notes.


### llm_calls table

In [40]:
# Get recent LLM calls
llm_calls = DB.query_all(
    "SELECT session_id, node_id, api_type, timestamp FROM llm_calls ORDER BY timestamp DESC LIMIT 20"
)

if llm_calls:
    df_llm = pd.DataFrame(llm_calls)
    print(f"Found {len(llm_calls)} recent LLM calls:")
    display(df_llm)
else:
    print("No LLM calls found")

2025-11-19 15:38:22,346 - ACO - INFO - [QUERY_ALL] START thread=140704361521024 sql=SELECT session_id, node_id, api_type, timestamp FROM llm_calls ORDER BY timestamp DESC LIMIT 20...
2025-11-19 15:38:22,347 - ACO - INFO - [CONN] GET conn=5044824304 thread=140704361521024 caller=postgres.py:query_all:226
2025-11-19 15:38:22,389 - ACO - INFO - [QUERY_ALL] SUCCESS thread=140704361521024 conn=5044824304 rows=0
2025-11-19 15:38:22,389 - ACO - INFO - [CONN] RETURN conn=5044824304 thread=140704361521024 caller=postgres.py:query_all:238


No LLM calls found


## Clear tables

Here's a function to clear all records from the experiments table:

In [24]:
def clear_experiments_table():
    """Clear all records from the experiments table (deletes all records)"""
    try:
        # Execute DELETE query to remove all records
        DB.execute("DELETE FROM llm_calls")
        DB.execute("DELETE FROM attachments")
        DB.execute("DELETE FROM experiments")

        print("✅ Successfully cleared all records from experiments table")
        
        # Verify the table is empty
        count = DB.query_one("SELECT COUNT(*) as count FROM experiments")
        print(f"   Remaining records: {count['count']}")
        
    except Exception as e:
        print(f"❌ Failed to clear experiments table: {e}")
        
# Call the function to clear the table
# Uncomment the line below to actually run it:
clear_experiments_table()

2025-11-19 14:34:36,782 - ACO - INFO - [EXECUTE] START thread=140704361521024 sql=DELETE FROM llm_calls...
2025-11-19 14:34:37,382 - ACO - DEBUG - Database schema initialized
2025-11-19 14:34:37,383 - ACO - INFO - Initialized PostgreSQL connection pool to workflow-postgres.cm14iy6021bi.us-east-1.rds.amazonaws.com
2025-11-19 14:34:37,384 - ACO - INFO - [CONN] GET conn=5044824304 thread=140704361521024 caller=postgres.py:execute:249
2025-11-19 14:34:37,436 - ACO - INFO - [EXECUTE] SUCCESS thread=140704361521024 conn=5044824304
2025-11-19 14:34:37,437 - ACO - INFO - [CONN] RETURN conn=5044824304 thread=140704361521024 caller=postgres.py:execute:261
2025-11-19 14:34:37,438 - ACO - INFO - [EXECUTE] START thread=140704361521024 sql=DELETE FROM attachments...
2025-11-19 14:34:37,439 - ACO - INFO - [CONN] GET conn=5044824304 thread=140704361521024 caller=postgres.py:execute:249
2025-11-19 14:34:37,490 - ACO - INFO - [EXECUTE] SUCCESS thread=140704361521024 conn=5044824304
2025-11-19 14:34:37,4

✅ Successfully cleared all records from experiments table
   Remaining records: 0


# Drop tables

In [ ]:
# FRESH CONNECTION + EMERGENCY RESET (All-in-one)
print("🚨 Creating fresh connection and performing emergency reset...")
from aco.common.constants import REMOTE_DATABASE_URL
import psycopg2
import psycopg2.extras
from urllib.parse import urlparse

if REMOTE_DATABASE_URL:
    parsed = urlparse(REMOTE_DATABASE_URL)
    
    try:
        # Create a fresh connection
        print("1️⃣ Establishing fresh connection...")
        fresh_conn = psycopg2.connect(
            host=parsed.hostname,
            port=parsed.port or 5432,
            user=parsed.username,
            password=parsed.password,
            database=parsed.path[1:],
            connect_timeout=15
        )
        fresh_conn.autocommit = True  # Auto-commit for immediate effect
        cursor = fresh_conn.cursor()
        print("   ✅ Fresh connection established")
        
        # Kill all other connections
        print("\n2️⃣ Killing hanging sessions...")
        cursor.execute("SELECT current_database()")
        current_db = cursor.fetchone()[0]
        
        cursor.execute("""
            SELECT pg_terminate_backend(pid) as killed
            FROM pg_stat_activity
            WHERE datname = %s 
            AND pid != pg_backend_pid()
            AND state != 'idle'
        """, (current_db,))
        killed_sessions = cursor.fetchall()
        print(f"   ✅ Terminated {len(killed_sessions)} hanging sessions")
        
        # Force drop all tables (uncomment below)
        print("\n3️⃣ Force dropping tables...")
        cursor.execute("DROP TABLE IF EXISTS llm_calls CASCADE")
        cursor.execute("DROP TABLE IF EXISTS attachments CASCADE")
        cursor.execute("DROP TABLE IF EXISTS experiments CASCADE")
        print("   ✅ All tables dropped")
        
        # Recreate experiments table
        print("\n4️⃣ Recreating tables with BYTEA schema...")
        cursor.execute("""
            CREATE TABLE experiments (
                session_id TEXT PRIMARY KEY,
                parent_session_id TEXT,
                graph_topology TEXT,
                color_preview TEXT,
                timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                cwd TEXT,
                command TEXT,
                environment TEXT,
                code_hash TEXT,
                name TEXT,
                success TEXT CHECK (success IN ('', 'Satisfactory', 'Failed')),
                notes TEXT,
                log TEXT,
                FOREIGN KEY (parent_session_id) REFERENCES experiments (session_id),
                UNIQUE (parent_session_id, name)
            )
        """)
        print("   ✅ Experiments table created")
        
        # Recreate llm_calls table with BYTEA output (THE CRITICAL FIX!)
        cursor.execute("""
            CREATE TABLE llm_calls (
                session_id TEXT,
                node_id TEXT,
                input BYTEA,
                input_hash TEXT,
                input_overwrite BYTEA,
                output BYTEA,
                color TEXT,
                label TEXT,
                api_type TEXT,
                timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                PRIMARY KEY (session_id, node_id)
            )
        """)
        # HACK: Remove foreign key constraint because things can happen in parallel.
        print("   ✅ LLM calls table created with BYTEA output column")
        
        # Recreate attachments table
        cursor.execute("""
            CREATE TABLE attachments (
                file_id TEXT PRIMARY KEY,
                session_id TEXT,
                line_no INTEGER,
                content_hash TEXT,
                file_path TEXT,
                taint TEXT,
                FOREIGN KEY (session_id) REFERENCES experiments (session_id)
            )
        """)
        print("   ✅ Attachments table created")
        
        # Create indexes
        cursor.execute("CREATE INDEX attachments_content_hash_idx ON attachments(content_hash)")
        cursor.execute("CREATE INDEX original_input_lookup ON llm_calls(session_id, input_hash)")
        cursor.execute("CREATE INDEX experiments_timestamp_idx ON experiments(timestamp DESC)")
        print("   ✅ Indexes created")
        
        # Verify the schema fix
        cursor.execute("""
            SELECT data_type 
            FROM information_schema.columns 
            WHERE table_name = 'llm_calls' AND column_name = 'output'
        """)
        output_type = cursor.fetchone()[0]
        
        print(f"\n🎉 SUCCESS! Schema completely rebuilt")
        print(f"   ✅ Output column is now: {output_type}")
        print(f"   ✅ All tables recreated cleanly")
        print(f"   ✅ All locks cleared")
                    
        fresh_conn.close()
        
    except Exception as e:
        print(f"❌ Emergency reset failed: {e}")
        print(f"   Error type: {type(e).__name__}")
        
        if "timeout" in str(e).lower():
            print("   → Connection still timing out. Database may be overloaded.")
        elif "permission" in str(e).lower():
            print("   → Permission denied. Check database user privileges.")
        else:
            print(f"   → Unexpected error: {e}")
            
else:
    print("❌ No DATABASE_URL found")

🚨 Creating fresh connection and performing emergency reset...
1️⃣ Establishing fresh connection...
   ✅ Fresh connection established

2️⃣ Killing hanging sessions...
   ✅ Terminated 0 hanging sessions

3️⃣ Force dropping tables...
   ✅ All tables dropped

4️⃣ Recreating tables with BYTEA schema...
   ✅ Experiments table created
   ✅ LLM calls table created with BYTEA output column
   ✅ Attachments table created
   ✅ Indexes created

🎉 SUCCESS! Schema completely rebuilt
   ✅ Output column is now: bytea
   ✅ All tables recreated cleanly
   ✅ All locks cleared

✨ The fix is complete! Now:
   1. Your original script should work without errors
   2. Revert the cache_manager.py change (use dill.loads directly)
   3. The web app should show clear data


In [ ]:
# Verify the schema change was successful
final_schema = DB.query_all(
    """
    SELECT column_name, data_type, is_nullable
    FROM information_schema.columns 
    WHERE table_name = 'llm_calls' 
    ORDER BY ordinal_position
    """
)

print("Final llm_calls table schema:")
for col in final_schema:
    status = ""
    if col['column_name'] == 'output':
        if col['data_type'] == 'bytea':
            status = " ✅"
        else:
            status = " ❌"
    print(f"  {col['column_name']}: {col['data_type']} ({'NULL' if col['is_nullable'] == 'YES' else 'NOT NULL'}){status}")

# Test that we can insert and retrieve binary data
print("\nTesting binary data storage...")
try:
    import dill
    test_data = {"test": "data", "number": 42}
    test_pickle = dill.dumps(test_data)
    
    # This should work now without error
    print(f"✅ Successfully created pickle data: {len(test_pickle)} bytes")
    print(f"✅ Can load pickle back: {dill.loads(test_pickle)}")
    print("Migration appears successful!")
    
except Exception as e:
    print(f"❌ Error with pickle test: {e}")

2025-11-17 02:34:14,429 - ACO - DEBUG - Database schema initialized
2025-11-17 02:34:14,430 - ACO - INFO - Initialized PostgreSQL connection to workflow-postgres.cm14iy6021bi.us-east-1.rds.amazonaws.com


Final llm_calls table schema:
  session_id: text (NOT NULL)
  node_id: text (NOT NULL)
  input: bytea (NULL)
  input_hash: text (NULL)
  input_overwrite: bytea (NULL)
  output: bytea (NULL) ✅
  color: text (NULL)
  label: text (NULL)
  api_type: text (NULL)
  timestamp: timestamp without time zone (NULL)

Testing binary data storage...
✅ Successfully created pickle data: 41 bytes
✅ Can load pickle back: {'test': 'data', 'number': 42}
Migration appears successful!


In [ ]:
def clear_experiments_table():
    """Clear all records from the experiments table with cascading delete"""
    try:
        # First delete from dependent tables (in order of dependencies)
        print("🗑️  Starting cascaded delete...")
        
        # 1. Delete from llm_calls (depends on experiments)
        DB.execute("DELETE FROM llm_calls")
        print("   ✅ Cleared llm_calls table")
        
        # 2. Delete from attachments (depends on experiments)
        DB.execute("DELETE FROM attachments")
        print("   ✅ Cleared attachments table")
        
        # 3. Finally delete from experiments (parent table)
        DB.execute("DELETE FROM experiments")
        print("   ✅ Cleared experiments table")
        
        # Verify all tables are empty
        exp_count = DB.query_one("SELECT COUNT(*) as count FROM experiments")
        llm_count = DB.query_one("SELECT COUNT(*) as count FROM llm_calls")
        att_count = DB.query_one("SELECT COUNT(*) as count FROM attachments")
        
        print(f"\n📊 Final counts:")
        print(f"   experiments: {exp_count['count']} records")
        print(f"   llm_calls: {llm_count['count']} records")
        print(f"   attachments: {att_count['count']} records")
        
        print("\n✨ Successfully cleared all tables!")
        
    except Exception as e:
        print(f"❌ Failed to clear tables: {e}")
        print("   Make sure to handle foreign key constraints")
        
# Call the function to clear all tables
# Uncomment the line below to actually run it:
# clear_experiments_table()